# iR Correction based on EIS

The EIS analysis is based on code by Frederik Johannes Stender.

This script helps you to set the `resistance` in multiple NOMAD entries depending on an EIS that is uploaded as a `CE_NOME_ElectrochemicalImpedanceSpectroscopy`. 

The correction is then **automatically** performed within the NOMAD entries: `voltage_ref_compensated = (voltage) - (current * resistance)`.

### 1) Select Sample and Environment IDs

In [1]:
%%capture
%matplotlib widget
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import numpy as np

import sys
sys.path.insert(1, '../../python-scripts-c6fxKDJrSsWp1xCxON1Y7g')
from api_calls import *
sys.path.insert(1, '../../nome-analysis-notebooks-1OmPPM5nSb-Y_g41AdQ66g')
from helpers import create_zip_download_link
from handle_connected_experiments import *
url = "https://nomad-hzb-ce.de/nomad-oasis/api/v1"
import os
token = os.environ['NOMAD_CLIENT_ACCESS_TOKEN']

In [2]:
def get_nomad_samples(my_ids_only=False):
    if my_ids_only:
        owner = 'user'
    else:
        owner = 'visible'
    data = get_all_entry_data_of_type(url, token, entry_type='CE_NOME_Sample', nomad_owner_type=owner, with_meta=False,)
    return data

def get_nomad_environments(my_ids_only=False):
    if my_ids_only:
        owner = 'user'
    else:
        owner = 'visible'
    data = get_all_entry_data_of_type(url, token, entry_type='CE_NOME_Environment', nomad_owner_type=owner, with_meta=False,)
    return data

def get_eis_df(nomad_eis_json):
    columns = pd.MultiIndex.from_tuples([
        ('time',        '[s]'),
        ('frequency',   '[Hz]'),
        ('z_real',      '[Ω]'),
        ('z_imaginary', '[Ω]'),
        ('|z|',         '[Ω"]'),
        ('phase',       '[°]'),
    ])

    # Frederik computes |z| like this, it seems to match z_modulus (come back to this code if z_modulus makes problems)
    # np.sqrt(np.array(nomad_eis_json.get('z_real'))**2 + np.array(nomad_eis_json.get('z_imaginary'))**2),

    data = [
        nomad_eis_json.get('time', []),
        nomad_eis_json.get('frequency', []),
        nomad_eis_json.get('z_real', []),
        nomad_eis_json.get('z_imaginary', []),
        nomad_eis_json.get('z_modulus', []),
        nomad_eis_json.get('z_angle', []),
    ]
    return pd.DataFrame(np.stack(data, axis=1), columns=columns)

In [3]:
def get_dropdown_text_options(data):
    return [('-- no ID selected --', {})] + [(f"{item.get('lab_id')} | {item.get('name')}", item) for item in data]

def get_eis_dropdown_text_options(eis_data):
    if not eis_data:
        return [('No EIS data found for given IDs', {'name': 'No EIS data found for given IDs'})]
    return [(f"{item.get('name')} | {item.get('data_file')}", item) for item in eis_data]

def get_matching_eis_data(sample_id, environment_entry_id, url, token):
    if sample_id is None:
        return [{'name': 'Please select a valid sampleID first.'}]
    full_eis_data = get_specific_data_of_sample(url, token, sample_id, 'Impedance', with_meta=True)
    eis_data = [{**data, **{'entry_id': metadata['entry_id'], 'upload_id': metadata['upload_id']}} for data, metadata in full_eis_data]
    if environment_entry_id:
        eis_data = [eis_entry for eis_entry in eis_data if environment_entry_id in eis_entry.get('environment', '')]
    return eis_data

def on_checkbox_change(change):
    my_ids_only = change['new']
    samples = get_nomad_samples(my_ids_only)
    environments = get_nomad_environments(my_ids_only)
    
    sample_dropdown.options = get_dropdown_text_options(samples)
    env_dropdown.options = get_dropdown_text_options(environments)

def on_id_dropdown_change(change):
    eis_selection_out.clear_output()
    eis_analysis_out.clear_output()
    lower_freq_limit_widget.value = ''
    upper_freq_limit_widget.value = ''
    eis_data = get_matching_eis_data(sample_dropdown.value.get('lab_id'), env_dropdown.value.get('entry_id'), url, token)
    eis_dropdown.options = get_eis_dropdown_text_options(eis_data)

def on_eis_dropdown_change(change):
    global eis_df
    with eis_selection_out:
        eis_selection_out.clear_output()
        eis_analysis_out.clear_output()
        lower_freq_limit_widget.value = ''
        upper_freq_limit_widget.value = ''
        eis_df = get_eis_df(eis_dropdown.value)
        display(eis_df)

own_ids_checkbox = widgets.Checkbox(
    value=False,
    description='just show my own IDs'
)

initial_samples = get_nomad_samples(own_ids_checkbox.value)
initial_envs = get_nomad_environments(own_ids_checkbox.value)
initial_eis = [{'name': 'Please select a valid sampleID first.'}]

eis_df = pd.DataFrame()

sample_dropdown = widgets.Dropdown(
    options=get_dropdown_text_options(initial_samples),
    description='NOME Sample:',
    layout=widgets.Layout(width='600px'),
    style={'description_width': 'initial'},
)

env_dropdown = widgets.Dropdown(
    options=get_dropdown_text_options(initial_envs),
    description='EnvironmentID:',
    layout=widgets.Layout(width='600px'),
    style={'description_width': 'initial'},
)

eis_dropdown = widgets.Dropdown(
    options=get_eis_dropdown_text_options(initial_eis),
    description='Please select the EIS file you want to analyse:',
    layout=widgets.Layout(width='600px'),
    style={'description_width': 'initial'},
)

eis_selection_out = widgets.Output()
save_eis_df_button = widgets.Button(
    description='Download table as csv',
    button_style='info',
    layout=widgets.Layout(width='auto'),
)
download_button_output = widgets.Output()
run_analysis_button = widgets.Button(
    description='Run EIS Analysis',
    button_style='info',
    layout=widgets.Layout(width='auto'),
)
eis_analysis_out = widgets.Output()

change_freq_lim_button = widgets.Button(description = 'Change Lims')
lower_freq_limit_widget = widgets.Text(description = 'Lower Frequency Limit', style={'description_width': 'initial'}, continuous_update=False)
upper_freq_limit_widget = widgets.Text(description = 'Upper Frequency Limit', style={'description_width': 'initial'}, continuous_update=False)

own_ids_checkbox.observe(on_checkbox_change, names='value')
sample_dropdown.observe(on_id_dropdown_change, names='value')
env_dropdown.observe(on_id_dropdown_change, names='value')
eis_dropdown.observe(on_eis_dropdown_change, names='value')

display(own_ids_checkbox, sample_dropdown, env_dropdown)

out = widgets.Output()
display(out)

Checkbox(value=False, description='just show my own IDs')

Dropdown(description='NOME Sample:', layout=Layout(width='600px'), options=(('-- no ID selected --', {}), ('CE…

Dropdown(description='EnvironmentID:', layout=Layout(width='600px'), options=(('-- no ID selected --', {}), ('…

Output()

In [4]:
# SampleID = 'CE-NOME_FrJo_FrSt_0095'
#sample_dropdown.value = 'CE-NOME_EICTRL_FrSt_0092 | CE-NOME EICTRL FrSt 0092'

#sample_dropdown.value = {
#    'entry_id': '-27oVGuYDY9Hw-lldx4x_BgEAfu_',
#    'm_def': 'nomad_chemical_energy.schema_packages.ce_nome_package.CE_NOME_Sample',
#    'name': 'CE-NOME EICTRL FrSt 0092',
#    'datetime': '2024-01-29T08:00:00+00:00',
#    'lab_id': 'CE-NOME_EICTRL_FrSt_0092',
#    'origin': 'Frederik Stender',
#    'chemical_composition_or_formulas': 'Au',
#    'component_description': 'Au Wire'
#}

### 2) Select the EIS entry you want to analyse

In [5]:
def on_eis_download_button_click(b):
    with download_button_output:
        download_button_output.clear_output()
        try:
            create_zip_download_link([eis_df], [eis_dropdown.value.get('data_file').replace('.DTA', '_eis_analysis.csv')], 'eis_table')
        except NameError:
            print('Please check above the button for a correct table before downloading.')

save_eis_df_button.on_click(on_eis_download_button_click)

display(eis_dropdown, eis_selection_out, save_eis_df_button, download_button_output)

Dropdown(description='Please select the EIS file you want to analyse:', layout=Layout(width='600px'), options=…

Output()

Button(button_style='info', description='Download table as csv', layout=Layout(width='auto'), style=ButtonStyl…

Output()

### 3) Inspect the analysis of the selected EIS entry

In [6]:
def get_eis_plot(eis_df, plt_title='EIS plot', x_label=('z_real', '[Ω]'), y1_label=('z_imaginary', '[Ω]'), y2_label=('phase', '[°]')):
    fig, ax1 = plt.subplots()
    
    color = 'tab:red'
    ax1.set_xlabel(' '.join(x_label))
    ax1.set_ylabel(' '.join(y1_label), color=color)
    ax1.plot(eis_df.get(x_label), eis_df.get(y1_label), 'o', color = color)
    ax1.tick_params(axis='y', labelcolor=color)

    ax2 = ax1.twinx()  # instantiate a second Axes that shares the same x-axis
    color = 'tab:blue'
    ax2.set_ylabel(' '.join(y2_label), color=color)  # we already handled the x-label with ax1
    ax2.plot(eis_df.get(x_label), eis_df.get(y2_label), 'o',color=color)
    ax2.tick_params(axis='y', labelcolor=color)

    axis_max = eis_df[[x_label, y1_label]].max().max()
    axis_min = eis_df[[x_label, y1_label]].min().min()
    ax1.set_ylim(axis_min - axis_max * 0.1, axis_max + axis_max * 0.1)
    ax1.set_xlim(axis_min - axis_max * 0.1, axis_max + axis_max * 0.1)
    ax2.set_ylim(-100, 0)
    #ax2.set_xlim(-100,300)

    plt.title(plt_title)
    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    return plt

def get_eis_values_for_phase_peak(eis_df, upper=False, lower=False):
    if not lower:
        lower = eis_df.get(('frequency', '[Hz]')).min()
    if not upper:
        lower = eis_df.get(('frequency', '[Hz]')).max()
    df_filtered = eis_df[(eis_df.get(('frequency', '[Hz]')) >= lower) & (eis_df.get(('frequency', '[Hz]')) <= upper)]
    return df_filtered.loc[df_filtered.get(('phase', '[°]')).abs().idxmin()]

def get_resistance_plot(eis_df, peak_df):
    x_label = ('frequency', '[Hz]')
    y1_label = ('z_real', '[Ω]')
    y2_label = ('phase', '[°]')

    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    ax1.set_xlabel(' '.join(x_label))
    ax1.set_ylabel(' '.join(y1_label), color='g')
    ax2.set_ylabel(' '.join(y2_label), color='b')
    ax1.set_xscale('log')
    ax1.set_yscale('log')

    ax1.plot(eis_df.get(x_label), eis_df.get(y1_label), color='g')
    ax1.plot(peak_df.get(x_label), peak_df.get(y1_label), 'xr')
    ax1.set_xlim(1, eis_df.get(x_label).max()*1.05)
    ax1.set_ylim(1, eis_df.get(y1_label).max()*1.05)
    ax2.plot(eis_df.get(x_label), eis_df.get(y2_label), color='b')
    ax2.plot(peak_df.get(x_label), peak_df.get(y2_label), 'xr')
    ax2.set_ylim(eis_df.get(y2_label).min()*1.05, eis_df.get(y2_label).max()*0.9)
    return plt

In [7]:
EISDict = {}

def perform_eis_analysis():
    global EISDict
    with eis_analysis_out:
        plt.close('all')
        eis_analysis_out.clear_output()
        plt_title = f'{eis_dropdown.value.get('name')} {eis_dropdown.value.get('data_file')}'
        eis_plt = get_eis_plot(eis_df, plt_title)
        eis_plt.show()
        phase_peak_df = get_eis_values_for_phase_peak(eis_df, float(upper_freq_limit_widget.value), float(lower_freq_limit_widget.value))
        resistance_plt = get_resistance_plot(eis_df, phase_peak_df)
        resistance_plt.show()
        print(f'Resistance: {phase_peak_df.get(('z_real', '[Ω]'))} Ω')
        
        EISDict[plt_title] = float(phase_peak_df.get(('z_real', '[Ω]')))

def on_eis_analysis_button_click(b):
    upper_freq_limit_widget.value = str(max(eis_df.get(('frequency', '[Hz]'))))
    lower_freq_limit_widget.value = str(min(eis_df.get(('frequency', '[Hz]'))))
    perform_eis_analysis()
    
def change_freq_limits_clicked(b):
    perform_eis_analysis()

run_analysis_button.on_click(on_eis_analysis_button_click)
change_freq_lim_button.on_click(change_freq_limits_clicked)

display(run_analysis_button, eis_analysis_out, lower_freq_limit_widget, upper_freq_limit_widget, change_freq_lim_button)

Button(button_style='info', description='Run EIS Analysis', layout=Layout(width='auto'), style=ButtonStyle())

Output()

Text(value='', continuous_update=False, description='Lower Frequency Limit', style=TextStyle(description_width…

Text(value='', continuous_update=False, description='Upper Frequency Limit', style=TextStyle(description_width…

Button(description='Change Lims', style=ButtonStyle())

### 2) In which uploads would you like to use this resistance for iR correction?

Please note: Only the uploads that you created yourself as main author are listed here.

Hold *shift* and *click* to select multiple uploads.

In [8]:
upload_list = get_all_owned_uploads(url, token, number_of_uploads=200)

options = [
    (f"{item['upload_id']} - {item.get('upload_name', '--no name given--')}", item['upload_id'])
    for item in upload_list
]

multi_select = widgets.SelectMultiple(
    options=options,
    description='Uploads:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%', height='150px')
)

upload_selection_output = widgets.Output()

def on_selection_change(change):
    with upload_selection_output:
        upload_selection_output.clear_output()
        selected_ids = change['new']  # list of selected upload_id
        if selected_ids:
            print("Selected Upload IDs:")
            for sid in selected_ids:
                print(sid)
        else:
            print("No Uploads selected.")

multi_select.observe(on_selection_change, names='value')

display(multi_select, upload_selection_output)

SelectMultiple(description='Uploads:', layout=Layout(height='150px', width='50%'), options=(('7_vy9XAZQYSnYpSO…

Output()

### 5) Apply the iR correction in NOMAD

The button below will associate all NOMAD entries of types `CE_NOME_Chronoamperometry`, `CE_NOME_Chronopotentiometry`, `CE_NOME_LinearSweepVoltammetry`, `CE_NOME_GalvanodynamicSweep`, and `CE_NOME_CyclicVoltammetry` with the calculated resistance.  

#### **Please note that this calibration process is not easily reversible.**
If you have other **Electrochemical Impedance Spectroscopy** entries linked in the **connected_experiments** in your NOMAD entries these will be overwritten.

In [9]:
def link_rhe_calibration(url, nomad_request_session, entry_id, resistance, eis_link):   
    return write_calibration_to_nomad(url, nomad_request_session, entry_id, 'data/resistance', resistance, eis_link)

def get_resistance():
    plt_title = f'{eis_dropdown.value.get('name')} {eis_dropdown.value.get('data_file')}'
    return EISDict.get(plt_title)

In [10]:
calibration_output = widgets.Output()

write_to_nomad_button = widgets.Button(
    description='Link resistance for iR correction',
    button_style='info',
    layout=widgets.Layout(width='auto'),
)

def on_write_to_nomad_button_click(b):
    with calibration_output:
        calibration_output.clear_output()
        new_resistance = get_resistance()
        if new_resistance is None:
            print('Please run the EIS analysis first to calculate the resistance.')
            return
        print('Please wait for the "All entries updated. DONE." at the bottom')
        print('-----')
        with requests.Session() as nomad_session:
            headers = {
                'Authorization': f'Bearer {token}',
                'Content-Type': 'application/json',
            }
            nomad_session.headers.update(headers)
            
            for upload_id in multi_select.value:
                entry_ids = get_nome_entryids_from_upload(url, nomad_session, upload_id)
                for entry_id in entry_ids:
                    method_type = 'Electrochemical Impedance Spectroscopy'
                    connected_entries_list = get_connected_experiments(url, nomad_session, entry_id)
                    new_eis_ref = get_reference_link(eis_dropdown.value)
                    same_method_list, other_refs_list = get_connected_experiments_filtered_by_method(url, nomad_session, connected_entries_list, method_type)
                    if same_method_list:
                        print(f'Entry {entry_id} already has connected experiments of the type {method_type}.')
                        print(f'The references to the following entry ids will be overwritten: {[get_entry_id_from_ref_link(entry_link) for entry_link in same_method_list]}')
                    other_refs_list.append(new_eis_ref)
                    link_rhe_calibration(url, nomad_session, entry_id, new_resistance, other_refs_list)
                    print(f'Use resistance of {new_resistance} V in NOMAD entry https://nomad-hzb-ce.de/nomad-oasis/gui/user/uploads/upload/id/{upload_id}/entry/id/{entry_id}')
                    print('-----')
        print('All entries updated. DONE.')

write_to_nomad_button.on_click(on_write_to_nomad_button_click)

display(write_to_nomad_button, calibration_output)

Button(button_style='info', description='Link resistance for iR correction', layout=Layout(width='auto'), styl…

Output()

### 6) Optional: Show overview of all resistances you analysed in this session

In [11]:
all_resistances_overview_button = widgets.Button(description = 'Show overview')
all_resistance_overview_output = widgets.Output()

def all_resistance_overview_clicked(b):
    with all_resistance_overview_output:
        all_resistance_overview_output.clear_output()
        x = [j for j in EISDict.keys()]
        y = [EISDict[i] for i in EISDict.keys()]
        
        #EISData = np.dstack((x,y))[0]
        #header = 'Water content\tResistance\n[%v/v]\t[\u03A9]\n \t '
        #print(EISData)
        plt.figure()
        plt.scatter(x, y)
        #plt.xlabel('Water content [%v/v]')
        plt.ylabel('Resistance [\u03A9]')
        plt.show()

all_resistances_overview_button.on_click(all_resistance_overview_clicked)

display(all_resistances_overview_button, all_resistance_overview_output)



Button(description='Show overview', style=ButtonStyle())

Output()